In [1]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers.flatten_observation import FlattenObservation
from gymnasium.wrappers.normalize import NormalizeObservation, NormalizeReward
import sailboat_gym
import gymnasium as gym
import numpy as np

2023-08-14 11:59:49.909725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 11:59:53.067288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
%env DOCKER_HOST=unix:///run/user/1512/docker.sock

env: DOCKER_HOST=unix:///run/user/1512/docker.sock


In [18]:
import pickle
import os.path as osp

class PersistentNormalizeObservation(NormalizeObservation):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print('PersistentNormalizeObservation init')
    
    def save(self, path: str) -> None:
        with open(f'{path}_normalize_obs.pkl', 'wb') as f:
            pickle.dump(self.obs_rms, f)

    def load(self, path: str) -> None:
        with open(f'{path}_normalize_obs.pkl', 'rb') as f:
            obs_rms = pickle.load(f)
        self.obs_rms = obs_rms

In [23]:
class Wrapper(gym.Wrapper):
    action_space = gym.spaces.Box(
        low=np.array([-1], dtype=np.float32),
        high=np.array([1], dtype=np.float32),
        shape=(1,),
        dtype=np.float32
    )

def create_env():
    env = gym.make('SailboatLSAEnv-v0')
    env = FlattenObservation(env)
    env = PersistentNormalizeObservation(env)
    env = Wrapper(env)
    print(env)
    return env

env = DummyVecEnv([create_env])
model = PPO('MlpPolicy', env, seed=0)

PersistentNormalizeObservation init
<Wrapper<PersistentNormalizeObservation<FlattenObservation<OrderEnforcing<PassiveEnvChecker<SailboatLSAEnv<SailboatLSAEnv-v0>>>>>>>


In [25]:
env = model.get_env()._get_target_envs(0)[0]
# env.env.obs_rms.mean += 1
# env.env.obs_rms.var += 1

In [28]:
print(env.env.obs_rms.mean, env.env.obs_rms.var)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [27]:
env.env.load("test")

In [29]:
a = model.get_env().get_attr('env', 0)[0].__class__

In [30]:
a

__main__.Wrapper

In [31]:
env = gym.make('SailboatLSAEnv-v0')
env = a(env)

In [32]:
env

<Wrapper<OrderEnforcing<PassiveEnvChecker<SailboatLSAEnv<SailboatLSAEnv-v0>>>>>

In [24]:
dir(model.get_env().get_attr('env', 0)[0])

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_space',
 '_cached_spec',
 '_is_protocol',
 '_metadata',
 '_np_random',
 '_observation_space',
 '_reward_range',
 'action_space',
 'class_name',
 'close',
 'env',
 'metadata',
 'np_random',
 'observation_space',
 'render',
 'render_mode',
 'reset',
 'reward_range',
 'spec',
 'step',
 'unwrapped',
 'wrapper_spec']